### Bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import shutil

### Diretórios

In [5]:
BASE_DIR = os.getcwd()
DIR_ARQUIVOS = os.path.join(BASE_DIR, "DADOS_METEREOLOGICOS_ESTADO_SP")
DIR_TRATADOS = os.path.join(BASE_DIR, "DADOS_TRATADOS")
DIR_TRATADOS_CIDADE_SP = os.path.join(BASE_DIR, 'DADOS_TRATADOS_CIDADE_SP')

### Filtro de dados

In [3]:
# Mapeamento das colunas antigas para as novas
COLUMN_MAPPING = {
    'DATA (YYYY-MM-DD)': 'Data',
    'HORA (UTC)': 'Hora UTC',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
    'RADIACAO GLOBAL (KJ/m²)': 'RADIACAO GLOBAL (Kj/m²)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'TEMPERATURA DO PONTO DE ORVALHO (°C)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
    'VENTO, RAJADA MAXIMA (m/s)': 'VENTO, RAJADA MAXIMA (m/s)',
    'VENTO, VELOCIDADE HORARIA (m/s)': 'VENTO, VELOCIDADE HORARIA (m/s)'
}

In [4]:
# Função para criar diretório se não existir
def ensure_directory(directory):
    """
    Verifica se o diretório existe e, caso contrário, cria-o.
    
    Args:
    directory (str): Caminho do diretório a ser verificado/criado.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Diretório criado: {directory}")
    else:
        print(f"Diretório já existe: {directory}")

In [5]:
def load_and_split(file_path):
    """
    Carrega a planilha e separa o cabeçalho extra dos dados principais.
    
    Args:
    file_path (str): Caminho para o arquivo CSV.
    
    Returns:
    tuple: DataFrames para o cabeçalho e os dados.
    """
    with open(file_path, 'r', encoding='latin1') as file:
        lines = file.readlines()

    header_lines = lines[:8]
    data_lines = lines[8:]

    df_header = pd.DataFrame([line.strip().split(';') for line in header_lines])
    column_names = data_lines[0].strip().split(';')
    df_data = pd.DataFrame([line.strip().split(';') for line in data_lines[1:]], columns=column_names)
    
        # Remover colunas vazias
    df_data = df_data.dropna(axis=1, how='all')
    try:
        df_data = df_data.drop(columns='')
    except:
        pass

    return df_header, df_data

In [6]:
def format_columns(df):
    """
    Renomeia as colunas de acordo com o mapeamento e faz as formatações necessárias.
    """
    df = df.rename(columns=COLUMN_MAPPING)
    
    # Verificar se a coluna 'Hora UTC' existe após o mapeamento
    if 'Hora UTC' in df.columns:
        # Remover qualquer sufixo de ":00" ou " UTC"
        df['Hora UTC'] = df['Hora UTC'].str.replace(' UTC', '').str.replace(':00', '')
        
        try:
            # Tentar converter usando o formato de horas e minutos
            df['Hora UTC'] = pd.to_datetime(df['Hora UTC'], format='%H%M').dt.time
        except ValueError:
            # Se falhar, tentar a conversão sem formato específico
            df['Hora UTC'] = pd.to_datetime(df['Hora UTC'], errors='coerce').dt.time
    
    return df


In [7]:
def format_numeric_columns(df):
    """
    Formata as colunas numéricas, substituindo vírgulas por pontos e convertendo para float.
    
    Args:
    df (DataFrame): DataFrame contendo os dados a serem formatados.
    
    Returns:
    DataFrame: DataFrame com colunas numéricas formatadas.
    """
    cols_to_convert = df.columns.difference(['Data', 'Hora UTC'])
    df[cols_to_convert] = df[cols_to_convert].replace(',', '.', regex=True)
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    return df

In [8]:
def save_to_excel(df_header, df_data, output_path):
    """
    Salva os DataFrames formatados em um arquivo Excel com diferentes abas.
    
    Args:
    df_header (DataFrame): DataFrame contendo o cabeçalho extra.
    df_data (DataFrame): DataFrame contendo os dados formatados.
    output_path (str): Caminho para salvar o arquivo Excel.
    """
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        df_header.to_excel(writer, sheet_name='Header', index=False, header=False)
        df_data.to_excel(writer, sheet_name='Data', index=False)

In [9]:
def process_file(file_path, output_path):
    """
    Processa o arquivo de entrada e o transforma em uma planilha formatada.
    
    Args:
    file_path (str): Caminho para o arquivo CSV de entrada.
    output_path (str): Caminho para salvar o arquivo Excel formatado.
    """
    df_header, df_data = load_and_split(file_path)
    df_data = format_columns(df_data)  # Renomear e formatar as colunas
    df_data = format_numeric_columns(df_data)
    save_to_excel(df_header, df_data, output_path)
    print(f"Arquivo Excel criado com sucesso em: {output_path}")

In [10]:
# Função principal para processar todos os arquivos .csv
def process_all_files():
    ensure_directory(DIR_TRATADOS)

    for year_dir in os.listdir(DIR_ARQUIVOS):
        year_path = os.path.join(DIR_ARQUIVOS, year_dir)
        
        if os.path.isdir(year_path):
            year_tratado_dir = os.path.join(DIR_TRATADOS, year_dir)
            ensure_directory(year_tratado_dir)

            for file_name in os.listdir(year_path):
                if file_name.endswith('.CSV'):
                    file_path = os.path.join(year_path, file_name)
                    output_file_name = file_name.replace('.CSV', '.xlsx')
                    output_path = os.path.join(year_tratado_dir, output_file_name)
                    
                    process_file(file_path, output_path)

In [11]:
# Executar a automação
process_all_files()

Diretório criado: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS
Diretório criado: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS\2001
Arquivo Excel criado com sucesso em: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS\2001\INMET_SE_SP_A705_BAURU_30-08-2001_A_31-12-2001.xlsx
Diretório criado: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS\2002
Arquivo Excel criado com sucesso em: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS\2002\INMET_SE_SP_A705_BAURU_01-01-2002_A_31-12-2002.xlsx
Arquivo Excel criado com sucesso em: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS\2002\INME

In [14]:
def separa_cidade_sp(DIR_TRATADOS, DIR_TRATADOS_CIDADE_SP):
    
    #Cria pasta DIR_TRATADOS_CIDADE_SP se não existir
    ensure_directory(DIR_TRATADOS_CIDADE_SP)
    
    # Percorrer todas as pastas e subpastas
    for subdir, _, files in os.walk(DIR_TRATADOS):
        for file in files:
            # Verificar se "SAO PAULO" está no nome do arquivo
            if "SAO PAULO" in file:
                # Caminho completo do arquivo
                caminho_arquivo = os.path.join(subdir, file)
                # Copiar o arquivo para a pasta de destino
                shutil.copy(caminho_arquivo, DIR_TRATADOS_CIDADE_SP)

    print("Arquivos copiados com sucesso!")

In [15]:
separa_cidade_sp(DIR_TRATADOS, DIR_TRATADOS_CIDADE_SP)

Diretório já existe: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP
Arquivos copiados com sucesso!


In [3]:
# Function to correct the 'Hora UTC' column
def correct_hora_utc(df):
    # Generate the correct sequence of times
    correct_times = pd.date_range("00:00", "23:00", freq="H").time
    
    # Replace the 'Hora UTC' column with the correct sequence
    # Since we might have more rows than needed, we need to handle it carefully
    df['Hora UTC'] = np.tile(correct_times, len(df) // len(correct_times) + 1)[:len(df)]
    
    return df

In [7]:
# Percorre todos os arquivos no diretório
for filename in os.listdir(DIR_TRATADOS_CIDADE_SP):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(DIR_TRATADOS_CIDADE_SP, filename)
        # Carrega o arquivo Excel
        xls = pd.ExcelFile(file_path)
        
        # Verifica se o arquivo possui a página 'Data'
        if 'Data' in xls.sheet_names:
            # Carrega a planilha 'Data'
            df = pd.read_excel(file_path, sheet_name='Data')
            
            # Verifica se há erros na coluna 'Hora UTC'
            if any(df['Hora UTC'].str.slice(3, 5) != '00'):
                # Corrige a coluna se houver erro
                df = correct_hora_utc(df)
                
                # Salva o arquivo corrigido
                corrected_file_path = os.path.join(DIR_TRATADOS_CIDADE_SP, f'Corrected_{filename}')
                with pd.ExcelWriter(corrected_file_path) as writer:
                    xls.parse('Header').to_excel(writer, sheet_name='Header', index=False)
                    df.to_excel(writer, sheet_name='Data', index=False)
                
                print(f'{filename} corrigido e salvo como {corrected_file_path}')

print("Processo concluído.")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50356\4096757400.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  correct_times = pd.date_range("00:00", "23:00", freq="H").time


INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2007_A_31-12-2007.xlsx corrigido e salvo como c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2007_A_31-12-2007.xlsx


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50356\4096757400.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  correct_times = pd.date_range("00:00", "23:00", freq="H").time


INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2009_A_31-12-2009.xlsx corrigido e salvo como c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2009_A_31-12-2009.xlsx


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50356\4096757400.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  correct_times = pd.date_range("00:00", "23:00", freq="H").time


...

INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2018_A_31-12-2018.xlsx corrigido e salvo como c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2018_A_31-12-2018.xlsx


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50356\4096757400.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  correct_times = pd.date_range("00:00", "23:00", freq="H").time


INMET_SE_SP_A701_SAO PAULO - MIRANTE_25-07-2006_A_31-12-2006.xlsx corrigido e salvo como c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_25-07-2006_A_31-12-2006.xlsx


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50356\4096757400.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  correct_times = pd.date_range("00:00", "23:00", freq="H").time


INMET_SE_SP_A771_SAO PAULO - INTERLAGOS_14-03-2018_A_31-12-2018.xlsx corrigido e salvo como c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A771_SAO PAULO - INTERLAGOS_14-03-2018_A_31-12-2018.xlsx
Processo concluído.
